In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install selenium

In [2]:
# !rm -v geckodriver.log_
# !mv -v geckodriver.log geckodriver.log_

In [3]:
import re
from datetime import datetime as dt

from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.options import FirefoxProfile

In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [5]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

url = 'https://www.avito.ru/sevastopol/kvartiry/prodam?'# '&pmax=5000000'

profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

In [6]:
class AvitoDownloader:
    
    def __init__(self,profile_path):
        self._options = Options()
        self._options.profile = FirefoxProfile(profile_path) 
        self._options.headless = True

    def load(self,url,is_debug=False):         
        driver = webdriver.Firefox(options=self._options)

        driver.get(url)
        html = driver.page_source
        root = BeautifulSoup(html)
        pages = self._get_pages_count(root)
        data = self._parse_page(root)
        
        if is_debug: 
            driver.quit() 
            return pd.DataFrame(data).dropna(),html 

        for p in tqdm(range(2,pages)):
            driver.get(url+f'&p={p}')
            root = BeautifulSoup(driver.page_source)
            data.extend( self._parse_page( root )  )

        driver.quit()    
        
        return pd.DataFrame(data).dropna()

    @classmethod
    def _parse_page(cls,root):
        return [ cls._parse_item(tag) for tag in root.find_all('div',{'data-marker':'item'}) ]
        
    @staticmethod
    def _parse_item(tag): return dict()

    @staticmethod
    def _get_pages_count(root):
        pp = re.sub( r'.*?p=', '', root.find_all('a',{'class':'pagination-page'})[-1].attrs['href'] ) 
        return 1 if not re.match(r'\d{1,3}', pp) else int(pp)    

In [7]:
class AvitoDownloaderRealty(AvitoDownloader):
    
    @classmethod
    def _parse_item(cls,tag):
        return {    
            'avito_id': tag.attrs['data-item-id'],   # https://www.avito.ru/<id>
            'title': cls._parse_item_tile(tag),
            'price': cls._parse_item_price(tag),
            'obj_name': cls._parse_item_dev_name(tag),
            'adr': cls._parse_item_adr(tag),
            'description': cls._parse_item_description(tag),
            }
        
    @staticmethod
    def _parse_item_tile(tag):
        try:
            return tag.find('a',attrs={'itemprop':'url'}).attrs['title']
        except:
            return ''    
      
    @staticmethod
    def _parse_item_price(tag):
        try:
            return tag.find('meta',attrs={'itemprop':'price'}).attrs['content']
        except:
            return ''
        
    @staticmethod
    def _parse_item_dev_name(tag):
        try:
            return tag.find('div',{'data-marker':'item-development-name'}).text
        except:
            return '' 
           
    @staticmethod
    def _parse_item_adr(tag):
        regex_address = re.compile('geo-address-.*')
        try:
            return tag.find('span',{'class':regex_address}).text
        except:
            return '' 
            
    @staticmethod
    def _parse_item_description(tag):
        regex_address = re.compile('geo-address-.*')
        try:
            return tag.find('meta',attrs={'itemprop':'description'}).attrs['content']
        except:
            return ''        
        
        
# 'cur':tag.find('meta',attrs={'itemprop':'priceCurrency'}).attrs['content'],
# tag.find('div',attrs={'data-marker':'item-address'}).text,       

----

In [8]:
# df,html = AvitoDownloaderRealty(profile_path).load(url,is_debug=True)
# df[['avito_id','title','price','adr','description',]].to_excel(f'page0_raw.xlsx',index=False)
# with open('page0.html','wt') as f: f.write( html)

In [9]:
# df

In [10]:
# # item-address
# # item-development-name
# root = BeautifulSoup(html)
# item = root.find_all('div',{'data-marker':'item'})[0]
# item.find('div',{'data-marker':'item-development-name'}).text
# # regex_address = re.compile('geo-address-.*')
# # item.find_all('span',{'class':regex_address})[0].text
# # [0].find('div',attrs={'data-marker':'item-address'}).text

---

In [11]:
%%time 

df = AvitoDownloaderRealty(profile_path).load(url)
ts = dt.now().strftime('%Y-%m-%d_%H-%M')
df[['avito_id','title','price','adr','description',]].to_excel(f'data/avito_{ts}_raw.xlsx',index=False)

/tmp/ipykernel_1251233/3756667461.py:5: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  self._options.profile = FirefoxProfile(profile_path)
/tmp/ipykernel_1251233/3756667461.py:5: DeprecationWarning: Setting a profile has been deprecated. Please use the set_preference and install_addons methods
  self._options.profile = FirefoxProfile(profile_path)


  0%|          | 0/78 [00:00<?, ?it/s]

CPU times: user 13.8 s, sys: 1.19 s, total: 15 s
Wall time: 5min 16s


In [13]:
# df

----

In [14]:
class AvitoDataCleanerRealty:
    
    @staticmethod
    def transform(data):
        df = data.copy()
        df['title'] = df['title'].str.lower().str.extract( r'.*«(.*)».*',expand=False)
        df['nrooms'] = df['title'].str.extract( r'.*(\d)-к. .*',expand=False)
        df['floor'] = df['title'].str.extract( r'.*(\d+)/\d+.эт.*',expand=False)
        df['nfloors'] = df['title'].str.extract( r'.*\d+/(\d+).эт.*',expand=False)
        df['area'] = (
            df['title']
            .str.extract( r'.*, ([\d,]+).*м²,.*',expand=False)
            .str.replace(',','.')
        )
        df['is_studio'] = df['title'].str.lower().str.match(r'.*студи.*')
        df['is_apartment'] = df['title'].str.lower().str.match(r'.*апартамент.*')
        df['is_part'] = df['title'].str.lower().str.match(r'.*дол.*')
        df['is_auction'] = df['title'].str.lower().str.match(r'.*аукци.*')
        df['is_openspace'] = df['title'].str.lower().str.match(r'.*своб.*планир.*')
        df['is_roof'] = df['description'].str.lower().str.match('.*мансард.*')
        df['is_SNT'] = (
            df['adr'].str.match('.*СТ .*') 
            | df['adr'].str.match('.*СНТ .*') 
            | df['adr'].str.lower().str.match('.*садов.*')
        )
        
        # df['is_fiolent'] = (
        #    df['description'].str.lower().str.match('.*фиолент.*')
        #    | df['adr'].str.lower().str.match('.*фиолент.*')
        #)

        # df[ df['adr'].str.match('.*СТ .*') ]
        # df[ df['adr'].str.match('.*СНТ .*') ]
        # df[ df['adr'].str.match('.*садов.*') ]
        # df[ df['description'].str.match('.*садов.*') ]

        # df['avito_id'] = df['avito_id'].astype(int)
        df['price'] = df['price'].astype(int)
        df['nrooms'] = df['nrooms'].fillna('0').astype(int)
        df['floor'] = df['floor'].fillna('0').astype(int)
        df['nfloors'] = df['nfloors'].fillna('0').astype(int)
        df['area'] = df['area'].fillna('0.').astype(float)
        df['priceM'] = df['price']/1e6
        df['is_last_floor'] = ( df['floor'] == df['nfloors'] )

        cols = [
         'adr',
         'obj_name',
         'title',
         'priceM',
         'nrooms',
         'floor',
         'nfloors',
         'area',
         'is_studio',
         'is_apartment',
         'is_part',
         'is_auction',
         'is_openspace',
         'is_SNT',
         # 'is_fiolent',   
         'is_last_floor', 
         'is_roof',   
         'description',
         'price',
         'avito_id',
        ]

        return df[cols]


In [15]:
df_ = AvitoDataCleanerRealty.transform(df)
df_

,adr,obj_name,title,priceM,nrooms,floor,nfloors,area,is_studio,is_apartment,is_part,is_auction,is_openspace,is_SNT,is_last_floor,is_roof,description,price,avito_id
0,"ул. Вакуленчука, 26",,"2-к. квартира, 70,7 м², 4/6 эт.",12.50,2,4,6,70.70,False,False,False,False,False,False,False,False,"Ваше время бесценно.\n\nКак Вы думаете — чем эта квартира примечательна? Ответ прост — Вы уже сейчас можете в ней жить. Она сделана для Вас! Вам не понадобится тратить время и средства на ремонт. Это одна из самых красивых квартир, которые Вы видели! Ее дела",12500000,2504072403
1,"ул. Военных Строителей, стр. 6.2",ЖК «Скифия»,"1-к. квартира, 47 м², 9/10 эт.",4.84,1,9,10,47.00,False,False,False,False,False,False,False,False,"Прекрасная квартира с городской пропиской в новом доме у моря. \n5 минут до пляжа! Остановка в шаговой доступности, 15 минут до центра города! \n\nДвухконтурный газовый котел. Индивидуальное отопление, которое позволит Вам не только сократить коммунальные пла",4835336,2276419738
2,"ш. Лабораторное, д. 33, секц. 6",ЖК «Новый»,"1-к. квартира, 41,8 м², 12/12 эт.",5.72,1,2,12,41.80,False,False,False,False,False,False,False,False,"В продаже однокомнатная квартира-бабочка на 12 этаже двенадцатиэтажного дома в новом комплексе жилых — ЖК «Новый» (корпус №6, вторая очередь), Нахимовский район города Севастополь. \n\nПодробнее о квартире: \n\n— Общая площадь квартиры: 41,76 м²;\n— Просторная",5722331,2397571906
3,"ул. Лётчиков, д. 10",Апарт-комплекс «GARDEN»,"1-к. апартаменты, 44,2 м², 2/5 эт.",9.82,1,2,5,44.20,False,True,False,False,False,False,False,False,"Давно мечтали жить у моря? \n\nСейчас самое время купить недвижимость в уникальном комплексе комфортабельных апартаментов! \n\nАпарт-комплекс «Garden» расположен на первой береговой линии побережья Черного моря, рядом с парком Победы, в самом экологически чист",9820000,2339710076
4,"ул. Военных Строителей, стр. 6.3",ЖК «Скифия»,"2-к. квартира, 62 м², 2/10 эт.",5.89,2,2,10,62.00,False,False,False,False,False,False,False,False,"Прекрасная квартира с городской пропиской в новом доме у моря. \n5 минут до пляжа! Остановка в шаговой доступности, 15 минут до центра города! \n\nДвухконтурный газовый котел. Индивидуальное отопление, которое позволит Вам не только сократить коммунальные пла",5885719,2276007945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,"пр-т Юрия Гагарина, 32",,"1-к. квартира, 30 м², 1/5 эт.",6.50,1,1,5,30.00,False,False,False,False,False,False,False,False,"Продается однокомнатная квартира на проспекте Юрия Гагарина,32! Отличное предложение ля небольшой семьи или в качестве высокорентабельной инвестиции. \n\nОбщая площадь квартиры 29 квадратных метров, кухня 6,3 метра, комната 17 метров, совмещенный сан. Узел,",6500000,2232124510
3949,"ул. Челнокова, д. 17, корп. 2",ЖК «Олимпия»,"1-к. квартира, 48 м², 2/5 эт.",12.10,1,2,5,48.00,False,False,False,False,False,False,False,False,"Продается крупногабаритная, совершенно новая однокомнатная квартира, расположенная в единственном в Севастополе жилом комплексе Премиум класса ЖК «Олимпия» по адресу ул. Челнокова, 17. Жилой комплекс находится в курортной части Севастополя, близость моря д",12100000,2460234791
3950,"Балаклава, ул. Аксютина, 34",,"1-к. квартира, 30,4 м², 4/5 эт.",7.00,1,4,5,30.40,False,False,False,False,False,False,False,False,"Однокомнатная квартира 30,4 м2, на 4/5 этаже дома. Расположена в г. Севастополь, Балаклавский район, ул. Аксютина, д. 34. \n\nКвартира подходит для комфортной жизни и отдыха, а также для удачной инвестиции. \n\nПозвоните нам, чтобы узнать все детали! \n\nТехниче",7000000,2489071310
3951,"Севастополь, Античный проспект, 26к3",,"1-к. квартира, 44 м², 9/10 эт.",8.00,1,9,10,44.00,False,False,False,False,False,False,False,False,"Предлагается уютная 1-но комнатная квартира с ремонтом на Античном! \n\nОбщая площадь 43,9 кв. М. (с учетом лоджии) по адресу пр. Античный д.26 корпус 3, в лучшем районе города в пешей доступности от моря! \n\nКвартира расположена н

In [16]:
ts = dt.now().strftime('%Y-%m-%d_%H-%M')
df_.to_excel(f'data/avito_{ts}.xlsx',index=False)

ts

'2022-08-31_17-43'

-----

In [ ]:
# df[ df['description'].str.match(r'.*часть дом.*') ][['title','description','is_part']]
# df[ df['description'].str.match(r'.*\d/\d квартир.*') ][['title','description','is_part']]

# df[ df['area'].isnull() ]
# df[ df['floor'].isnull() ]
# df[ df['nfloors'].isnull() ]
# df.query('is_openspace')

In [ ]:
# def get_pages_count(root):
#     pp = re.sub( r'.*?p=', '', root.find_all('a',{'class':'pagination-page'})[-1].attrs['href'] ) 
#     return 1 if not re.match(r'\d{1,3}', pp) else int(pp)


# def parse_item(tag):
#     try:
#         return {    
#     'avito_id':tag.attrs['data-item-id'],   # https://www.avito.ru/<id>
#     'title':tag.find('a',attrs={'itemprop':'url'}).attrs['title'],    
#     'price':tag.find('meta',attrs={'itemprop':'price'}).attrs['content'],
#      # 'cur':tag.find('meta',attrs={'itemprop':'priceCurrency'}).attrs['content'],
#     'adr':tag.find('div',attrs={'data-marker':'item-address'}).text,
#     'description':tag.find('meta',attrs={'itemprop':'description'}).attrs['content'],
#         }
#     except:
#         return dict()


# options=Options()
# options.profile = FirefoxProfile(profile_path) 
# options.headless = True
# driver = webdriver.Firefox( options=options)

# driver.get(url)
# html = driver.page_source

# root = BeautifulSoup(html)
# pages = get_pages_count(root)
# print(pages)
# data = [ parse_item(tag) for tag in root.find_all('div',{'data-marker':'item'}) ]

# for p in tqdm(range(2,pages)):
#     driver.get(url+f'&p={p}')
#     html = driver.page_source
#     root = BeautifulSoup(html)
#     data.extend( [ parse_item(tag) for tag in root.find_all('div',{'data-marker':'item'}) ] )

# # driver.close()
# driver.quit()

In [ ]:
# import pandas as pd
# df = pd.DataFrame(data).dropna()
# print(len(df))
# df.sample(10)